In [14]:
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Conv2D, Activation
from tensorflow.keras.layers import BatchNormalization, Input
from tensorflow.keras.backend import concatenate
import tensorflow as tf
from official.vision.image_classification.efficientnet import efficientnet_model



def layer1_multistream(res_x, res_y, num_cams, filter_num):
    """
    Multi-stream layer: Conv - ReLU - Conv - ReLU - BN

    :param res_x:
    :param res_y:
    :param num_cams:
    :param filter_num:
    :return:
    """
    if not hasattr(layer1_multistream, "instance"):
        layer1_multistream.instance = 0
    j = layer1_multistream.instance
    seq = Sequential()
    seq.add(Conv2D(filter_num, (3, 3), input_shape=(res_x, res_y, num_cams),
                   padding='valid', name=f'S1_C10_{j}', activation='relu'))
    seq.add(Conv2D(filter_num, (3, 3), input_shape=(res_x-2, res_y-2, 70),
                   padding='valid', name=f'S1_C20_{j}'))
    seq.add(BatchNormalization(axis=-1, name=f'S1_BN0_{j}'))
    seq.add(Activation('relu'))

    seq.add(Conv2D(filter_num, (3, 3), input_shape=(res_x-4, res_y-4, 70),
                   padding='valid', name=f'S1_C11_{j}', activation='relu'))
    seq.add(Conv2D(filter_num, (3, 3), input_shape=(res_x-6, res_y-6, 70),
                   padding='valid', name=f'S1_C21_{j}'))
    seq.add(BatchNormalization(axis=-1, name=f'S1_BN1_{j}'))
    seq.add(Activation('relu'))

    seq.add(Conv2D(filter_num, (3, 3), input_shape=(res_x-8, res_y-8, 70),
                   padding='valid', name=f'S1_C12_{j}', activation='relu'))
    seq.add(Conv2D(filter_num, (3, 3), input_shape=(res_x-10, res_y-10, 70),
                   padding='valid', name=f'S1_C22_{j}'))
    seq.add(BatchNormalization(axis=-1, name=f'S1_BN2_{j}'))
    seq.add(Activation('relu'))
    layer1_multistream.instance += 1
    return seq


def efficientnet():
    """
    Merged layer: Conv - ReLU - Conv - ReLU - BN

    :return: seq:
    """
    block_config = efficientnet_model.BlockConfig()
    blocks =  (# (input_filters, output_filters, kernel_size, num_repeat,
              #  expand_ratio, strides, se_ratio)
              # pylint: disable=bad-whitespace
              block_config.from_args(32, 16, 3, 1, 1, (1, 1), 0.25),
              block_config.from_args(16, 24, 3, 2, 6, (2, 2), 0.25),
              block_config.from_args(24, 40, 5, 2, 6, (2, 2), 0.25),
              block_config.from_args(40, 80, 3, 3, 6, (2, 2), 0.25),
              block_config.from_args(80, 112, 5, 3, 6, (1, 1), 0.25),
              block_config.from_args(112, 192, 5, 4, 6, (2, 2), 0.25),
              block_config.from_args(192, 320, 3, 1, 6, (1, 1), 0.25),
             )
      
    seq = efficientnet_model.EfficientNet(overrides={'num_classes': 3, 'input_channels': 140,
                                                     'rescale_input': False, 'blocks':blocks,
                                                     'width_coefficient':2.0})
    return seq


def define_epidef(sz_input1, sz_input2, view_n, filter_num):
    """
    Compiles the full network.

    :param sz_input1: resX
    :param sz_input2: resY
    :param view_n: num_cams
    :param filter_num: number of channels in multistream layers
    :return:
    """
    # 2-Input: Conv - ReLU - Conv - ReLU - BN
    input_stack_vert = Input(shape=(sz_input1, sz_input2, view_n), name='input_stack_vert')
    input_stack_hori = Input(shape=(sz_input1, sz_input2, view_n), name='input_stack_hori')

    # 2-Stream layer: Conv - ReLU - Conv - ReLU - BN
    mid_vert = layer1_multistream(sz_input1, sz_input2, view_n, filter_num)(input_stack_vert)
    mid_hori = layer1_multistream(sz_input1, sz_input2, view_n, filter_num)(input_stack_hori)

    # Merge layers
    mid_merged = concatenate([mid_vert, mid_hori])
    mid_merged_ = efficientnet()

    output = mid_merged_(mid_merged)
    model_512 = Model(inputs=[input_stack_vert, input_stack_hori], outputs=[output])
    metrics = ['accuracy',
               tf.keras.metrics.Precision(name='precision'),
               tf.keras.metrics.Recall(name='recall')]
    model_512.compile(loss='categorical_crossentropy', optimizer='adam', metrics=metrics)
    model_512.summary()
    return model_512


In [12]:
model = define_epidef(236, 236, 7, 70)

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_stack_vert (InputLayer)   [(None, 236, 236, 7) 0                                            
__________________________________________________________________________________________________
input_stack_hori (InputLayer)   [(None, 236, 236, 7) 0                                            
__________________________________________________________________________________________________
sequential_8 (Sequential)       (None, 224, 224, 70) 226170      input_stack_vert[0][0]           
__________________________________________________________________________________________________
sequential_9 (Sequential)       (None, 224, 224, 70) 226170      input_stack_hori[0][0]           
_______________________________________________________________________________________

In [15]:
import os
import datetime
import numpy as np

import tensorflow as tf
from tensorflow import keras
from epidef_fun.util import get_list_IDs
from epidef_fun.DataGenerator import DataGenerator


network_name = 'EPIDef_train'
iter00 = 0
load_weights = False
"""
Model parameters:
    first layer:  3 convolutional blocks
    second layer: 6 convolutional blocks
    last layer:   1 dense block?
"""
model_filter_number = 70
model_learning_rate = 1e-5
batch_size = 1
input_res = 236

# Define directory for saving checkpoint files:
directory_ckp = f"epidef_checkpoints\\{network_name}_ckp"
if not os.path.exists(directory_ckp):
    os.makedirs(directory_ckp)
if not os.path.exists('epidef_output\\'):
    os.makedirs('epidef_output\\')
directory_t = f"epidef_output\\{network_name}"
if not os.path.exists(directory_t):
    os.makedirs(directory_t)
# txt_name = f"epidef_checkpoints\\lf_{network_name}.txt"

# Load training data from lightfield .png files:
print("Loading lightfield paths...")
# dir_lf_images = ("C:\\Users\\muell\\Google Drive\\University\\Master_Project"
#                  + "\\data_storage\\lightfields")
dir_lf_images = ("C:\\Users\\muell\\Desktop\\blender_output_tmp")
list_IDs = get_list_IDs(dir_lf_images)[:100]

print("Done loading lightfield paths.")
fraction = np.int(len(list_IDs)*0.7)
list_IDs_train, list_IDs_test = list_IDs[:fraction], list_IDs[fraction:]

model = define_epidef(input_res, input_res, 7, model_filter_number)



generator_train = DataGenerator(list_IDs_train, batch_size=batch_size, train=False)
generator_test = DataGenerator(list_IDs_test, batch_size=batch_size, train=False)

# checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("epidef_model.h5", save_best_only=True)
# early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
#     callbacks = [checkpoint_cb]  # , keras.callbacks.TensorBoard(log_dir='./logs')]
# Try this out at some point:
# def exponential_decay(lr0, s):
#     def exponential_decay_fn(epoch):
#         return lr0 * 0.1**(epoch/s)
#     return exponential_decay_fn
# exponential_decay_fn = exponential_decay(0.01, 20)
# lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)
model.fit(generator_train, epochs=200, max_queue_size=10, initial_epoch=iter00, verbose=2,
          validation_data=generator_test)


print("Weights saved.")


Loading lightfield paths...
Good: 0.551051051051051
Scratch: 0.22447447447447447
Dent: 0.22447447447447447
Error: 0.0
Done loading lightfield paths.
Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_stack_vert (InputLayer)   [(None, 236, 236, 7) 0                                            
__________________________________________________________________________________________________
input_stack_hori (InputLayer)   [(None, 236, 236, 7) 0                                            
__________________________________________________________________________________________________
sequential_12 (Sequential)      (None, 224, 224, 70) 226170      input_stack_vert[0][0]           
__________________________________________________________________________________________________
sequential_13 (Sequential)      (None

KeyboardInterrupt: 